# Standard Keras Optimizers w/ Cyclic Learning Rate

In [1]:
from clr_optimizers import *

import numpy as np

Using Theano backend.
Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5110)
/home/bckenstler/anaconda3/envs/py36/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## Toy CLR Function

In [2]:
def triangular(iterations):
    """
    Test CLR function w/ following params:
    """
    step_size = 2000.
    lr = 0.01
    max_lr = 0.06
    cycle = np.floor(1+iterations/(2*step_size))
    x = np.abs(iterations/step_size - 2*cycle + 1)
    return lr + (max_lr-lr)*np.maximum(0, (1-x))

In [5]:
for iteration in range(1000, 11000, 1000):
    print(f"Learning rate at iteration {iteration}: {triangular(iteration)}")

Learning rate at iteration 1000: 0.034999999999999996
Learning rate at iteration 2000: 0.06
Learning rate at iteration 3000: 0.034999999999999996
Learning rate at iteration 4000: 0.01
Learning rate at iteration 5000: 0.034999999999999996
Learning rate at iteration 6000: 0.06
Learning rate at iteration 7000: 0.034999999999999996
Learning rate at iteration 8000: 0.01
Learning rate at iteration 9000: 0.034999999999999996
Learning rate at iteration 10000: 0.06


In [6]:
def triangular2(iterations):
    """
    Test CLR function w/ following params:
    """
    step_size = 2000.
    lr = 0.01
    max_lr = 0.06
    cycle = np.floor(1+iterations/(2*step_size))
    x = np.abs(iterations/step_size - 2*cycle + 1)
    return lr + (max_lr-lr)*np.maximum(0, (1-x))/float(2**(cycle-1))

In [7]:
for iteration in range(1000, 11000, 1000):
    print(f"Learning rate at iteration {iteration}: {triangular2(iteration)}")

Learning rate at iteration 1000: 0.034999999999999996
Learning rate at iteration 2000: 0.06
Learning rate at iteration 3000: 0.034999999999999996
Learning rate at iteration 4000: 0.01
Learning rate at iteration 5000: 0.0225
Learning rate at iteration 6000: 0.034999999999999996
Learning rate at iteration 7000: 0.0225
Learning rate at iteration 8000: 0.01
Learning rate at iteration 9000: 0.01625
Learning rate at iteration 10000: 0.0225


In [14]:
def exp_range(iterations):
    """
    Test CLR function w/ following params:
    """
    gamma=0.99994
    step_size = 2000.
    lr = 0.01
    max_lr = 0.06
    cycle = np.floor(1+iterations/(2*step_size))
    x = np.abs(iterations/step_size - 2*cycle + 1)
    return lr + (max_lr-lr)*np.maximum(0, (1-x))*gamma**(iterations)

In [15]:
for iteration in range(1000, 11000, 1000):
    print(f"Learning rate at iteration {iteration}: {exp_range(iteration)}")

Learning rate at iteration 1000: 0.0335440709585463
Learning rate at iteration 2000: 0.05434586218408506
Learning rate at iteration 3000: 0.030881642519600273
Learning rate at iteration 4000: 0.01
Learning rate at iteration 5000: 0.02852028882703049
Learning rate at iteration 6000: 0.044883439545310246
Learning rate at iteration 7000: 0.02642596351865889
Learning rate at iteration 8000: 0.01
Learning rate at iteration 9000: 0.02456847028878512
Learning rate at iteration 10000: 0.03744008785893043


## Test Model

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input

In [3]:
inp = Input(shape=(15,))                
x = Dense(10, activation='relu')(inp)
x = Dense(1, activation='sigmoid')(x)

model = Model(inp, x)

In [4]:
clr1 = {
    'mode':'triangular',
    'max_lr': 0.06,
    'step_size': 2000
}

In [5]:
clr2 = {
    'mode':'triangular2',
    'max_lr': 0.06,
    'step_size': 2000
}

In [6]:
clr3 = {
    'mode':'exp_range',
    'max_lr': 0.06,
    'step_size': 2000,
    'clr_gamma':0.99994
}

In [7]:
X = np.random.rand(2000000,15)

In [8]:
Y = np.random.randint(0,2,size=2000000)

## CLR Tests

Set batch size so number of iterations in one epoch is 1000, half the default step_size (2000).

Checked for each model. Note that "current_lr" is updated immediately following cycle update, and should follow expectations. It would only be different if rate-decay is in effect, as this happens prior to cycle.

Tested each CLR implementation and compare with toy results

### SGD

In [9]:
model.compile(optimizer=SGD(lr=0.01,clr=clr1), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


In [10]:
model.compile(optimizer=SGD(lr=0.01,clr=clr2), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.022487498819828033
Number of iterations: 6000.0
Current LR: 0.03498750180006027
Number of iterations: 7000.0
Current LR: 0.022512491792440414
Number of iterations: 8000.0
Current LR: 0.010012492537498474
Number of iterations: 9000.0
Current LR: 0.01624375209212303
Number of iterations: 10000.0
Current LR: 0.02249375358223915


In [11]:
model.compile(optimizer=SGD(lr=0.01,clr=clr3), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.033522482961416245
Number of iterations: 2000.0
Current LR: 0.054328400641679764
Number of iterations: 3000.0
Current LR: 0.03090522810816765
Number of iterations: 4000.0
Current LR: 0.010019666515290737
Number of iterations: 5000.0
Current LR: 0.028505021706223488
Number of iterations: 6000.0
Current LR: 0.044872939586639404
Number of iterations: 7000.0
Current LR: 0.026446033269166946
Number of iterations: 8000.0
Current LR: 0.010015464387834072
Number of iterations: 9000.0
Current LR: 0.024557817727327347
Number of iterations: 10000.0
Current LR: 0.037434376776218414


### RMSProp

In [12]:
model.compile(optimizer=RMSprop(lr=0.01,clr=clr1), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


In [13]:
model.compile(optimizer=RMSprop(lr=0.01,clr=clr2), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.022487498819828033
Number of iterations: 6000.0
Current LR: 0.03498750180006027
Number of iterations: 7000.0
Current LR: 0.022512491792440414
Number of iterations: 8000.0
Current LR: 0.010012492537498474
Number of iterations: 9000.0
Current LR: 0.01624375209212303
Number of iterations: 10000.0
Current LR: 0.02249375358223915


In [14]:
model.compile(optimizer=RMSprop(lr=0.01,clr=clr3), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.033522482961416245
Number of iterations: 2000.0
Current LR: 0.054328400641679764
Number of iterations: 3000.0
Current LR: 0.03090522810816765
Number of iterations: 4000.0
Current LR: 0.010019666515290737
Number of iterations: 5000.0
Current LR: 0.028505021706223488
Number of iterations: 6000.0
Current LR: 0.044872939586639404
Number of iterations: 7000.0
Current LR: 0.026446033269166946
Number of iterations: 8000.0
Current LR: 0.010015464387834072
Number of iterations: 9000.0
Current LR: 0.024557817727327347
Number of iterations: 10000.0
Current LR: 0.037434376776218414


### Adagrad

In [15]:
model.compile(optimizer=Adagrad(lr=0.01,clr=clr1), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


In [16]:
model.compile(optimizer=Adagrad(lr=0.01,clr=clr2), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.022487498819828033
Number of iterations: 6000.0
Current LR: 0.03498750180006027
Number of iterations: 7000.0
Current LR: 0.022512491792440414
Number of iterations: 8000.0
Current LR: 0.010012492537498474
Number of iterations: 9000.0
Current LR: 0.01624375209212303
Number of iterations: 10000.0
Current LR: 0.02249375358223915


In [17]:
model.compile(optimizer=Adagrad(lr=0.01,clr=clr3), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.033522482961416245
Number of iterations: 2000.0
Current LR: 0.054328400641679764
Number of iterations: 3000.0
Current LR: 0.03090522810816765
Number of iterations: 4000.0
Current LR: 0.010019666515290737
Number of iterations: 5000.0
Current LR: 0.028505021706223488
Number of iterations: 6000.0
Current LR: 0.044872939586639404
Number of iterations: 7000.0
Current LR: 0.026446033269166946
Number of iterations: 8000.0
Current LR: 0.010015464387834072
Number of iterations: 9000.0
Current LR: 0.024557817727327347
Number of iterations: 10000.0
Current LR: 0.037434376776218414


### Adadelta

In [18]:
model.compile(optimizer=Adadelta(lr=0.01,clr=clr1), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


In [19]:
model.compile(optimizer=Adadelta(lr=0.01,clr=clr2), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.022487498819828033
Number of iterations: 6000.0
Current LR: 0.03498750180006027
Number of iterations: 7000.0
Current LR: 0.022512491792440414
Number of iterations: 8000.0
Current LR: 0.010012492537498474
Number of iterations: 9000.0
Current LR: 0.01624375209212303
Number of iterations: 10000.0
Current LR: 0.02249375358223915


In [20]:
model.compile(optimizer=Adadelta(lr=0.01,clr=clr3), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.033522482961416245
Number of iterations: 2000.0
Current LR: 0.054328400641679764
Number of iterations: 3000.0
Current LR: 0.03090522810816765
Number of iterations: 4000.0
Current LR: 0.010019666515290737
Number of iterations: 5000.0
Current LR: 0.028505021706223488
Number of iterations: 6000.0
Current LR: 0.044872939586639404
Number of iterations: 7000.0
Current LR: 0.026446033269166946
Number of iterations: 8000.0
Current LR: 0.010015464387834072
Number of iterations: 9000.0
Current LR: 0.024557817727327347
Number of iterations: 10000.0
Current LR: 0.037434376776218414


### Adam

In [21]:
model.compile(optimizer=Adam(lr=0.01,clr=clr1), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


In [22]:
model.compile(optimizer=Adam(lr=0.01,clr=clr2), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.022487498819828033
Number of iterations: 6000.0
Current LR: 0.03498750180006027
Number of iterations: 7000.0
Current LR: 0.022512491792440414
Number of iterations: 8000.0
Current LR: 0.010012492537498474
Number of iterations: 9000.0
Current LR: 0.01624375209212303
Number of iterations: 10000.0
Current LR: 0.02249375358223915


In [23]:
model.compile(optimizer=Adam(lr=0.01,clr=clr3), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.033522482961416245
Number of iterations: 2000.0
Current LR: 0.054328400641679764
Number of iterations: 3000.0
Current LR: 0.03090522810816765
Number of iterations: 4000.0
Current LR: 0.010019666515290737
Number of iterations: 5000.0
Current LR: 0.028505021706223488
Number of iterations: 6000.0
Current LR: 0.044872939586639404
Number of iterations: 7000.0
Current LR: 0.026446033269166946
Number of iterations: 8000.0
Current LR: 0.010015464387834072
Number of iterations: 9000.0
Current LR: 0.024557817727327347
Number of iterations: 10000.0
Current LR: 0.037434376776218414


### Adamax

In [24]:
model.compile(optimizer=Adamax(lr=0.01,clr=clr1), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


In [25]:
model.compile(optimizer=Adamax(lr=0.01,clr=clr2), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.022487498819828033
Number of iterations: 6000.0
Current LR: 0.03498750180006027
Number of iterations: 7000.0
Current LR: 0.022512491792440414
Number of iterations: 8000.0
Current LR: 0.010012492537498474
Number of iterations: 9000.0
Current LR: 0.01624375209212303
Number of iterations: 10000.0
Current LR: 0.02249375358223915


In [26]:
model.compile(optimizer=Adamax(lr=0.01,clr=clr3), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.033522482961416245
Number of iterations: 2000.0
Current LR: 0.054328400641679764
Number of iterations: 3000.0
Current LR: 0.03090522810816765
Number of iterations: 4000.0
Current LR: 0.010019666515290737
Number of iterations: 5000.0
Current LR: 0.028505021706223488
Number of iterations: 6000.0
Current LR: 0.044872939586639404
Number of iterations: 7000.0
Current LR: 0.026446033269166946
Number of iterations: 8000.0
Current LR: 0.010015464387834072
Number of iterations: 9000.0
Current LR: 0.024557817727327347
Number of iterations: 10000.0
Current LR: 0.037434376776218414


### Nadam

In [27]:
model.compile(optimizer=Nadam(lr=0.01,clr=clr1), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.034974999725818634
Number of iterations: 6000.0
Current LR: 0.059974998235702515
Number of iterations: 7000.0
Current LR: 0.035024985671043396
Number of iterations: 8000.0
Current LR: 0.01002498622983694
Number of iterations: 9000.0
Current LR: 0.03497501090168953
Number of iterations: 10000.0
Current LR: 0.05997500941157341


In [28]:
model.compile(optimizer=Nadam(lr=0.01,clr=clr2), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.034974999725818634
Number of iterations: 2000.0
Current LR: 0.059974998235702515
Number of iterations: 3000.0
Current LR: 0.03502499684691429
Number of iterations: 4000.0
Current LR: 0.01002499833703041
Number of iterations: 5000.0
Current LR: 0.022487498819828033
Number of iterations: 6000.0
Current LR: 0.03498750180006027
Number of iterations: 7000.0
Current LR: 0.022512491792440414
Number of iterations: 8000.0
Current LR: 0.010012492537498474
Number of iterations: 9000.0
Current LR: 0.01624375209212303
Number of iterations: 10000.0
Current LR: 0.02249375358223915


In [29]:
model.compile(optimizer=Nadam(lr=0.01,clr=clr3), loss = 'binary_crossentropy', metrics=['accuracy'])
for i in range(0,10):
    model.fit(X, Y, batch_size=2000, nb_epoch=1, verbose=0)
    config = model.optimizer.get_config()
    print(f'Number of iterations: {config["clr_iterations"]}')
    print(f'Current LR: {config["current_lr"]}')

Number of iterations: 1000.0
Current LR: 0.033522482961416245
Number of iterations: 2000.0
Current LR: 0.054328400641679764
Number of iterations: 3000.0
Current LR: 0.03090522810816765
Number of iterations: 4000.0
Current LR: 0.010019666515290737
Number of iterations: 5000.0
Current LR: 0.028505021706223488
Number of iterations: 6000.0
Current LR: 0.044872939586639404
Number of iterations: 7000.0
Current LR: 0.026446033269166946
Number of iterations: 8000.0
Current LR: 0.010015464387834072
Number of iterations: 9000.0
Current LR: 0.024557817727327347
Number of iterations: 10000.0
Current LR: 0.037434376776218414


Values check out